In [ ]:
import os

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import r2_score
from scipy.stats import pearsonr

In [ ]:
df = pd.read_csv('/home/oncopole/Documents/Rudy/DeepOncopole/data/DB_PATH_NIFTI.csv')
df = df[df['PET']=='pet0']

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
import numpy as np
import SimpleITK as sitk

dtypes = {'pet_img': sitk.sitkFloat32,
          'ct_img': sitk.sitkFloat32,
          'mask_img': sitk.sitkUInt8}

# sitk.ReadImage(filename, self.dtypes['pet_img'])
# sitk.GetArrayFromImage()

In [ ]:
from skimage import filters

def calculate_threshold(roi, threshold, tval):
    if threshold == 'absolute':
        if tval is None:
            return 2.5
        return tval
    elif threshold == 'relative':
        if len(roi) == 0:
            return 1.0
        else:
            try : 
                SUV_max = np.max(roi)
            except:
                SUV_max = 0.0
            if tval is None:
                return SUV_max * 0.42
            else:
                return SUV_max * tval
    
    elif threshold == 'otsu':
        try :
            return filters.threshold_otsu(roi)
        except:
            if len(roi) == 0 :
                return 0.0
            else:
                return np.max(roi) + 1


def roi2mask(pet_array, mask_array, threshold='otsu', tval=None):
    if len(mask_array.shape)==3:
        mask_array = np.expand_dims(mask_array, axis=0)
    
    new_mask = np.zeros(mask_array.shape[1:], dtype=np.int8)
    
    for num_slice in range(mask_array.shape[0]):
        mask_slice = mask_array[num_slice]
        roi = pet_array[mask_slice > 0]
        
        t = calculate_threshold(roi, threshold=threshold, tval=tval)

        # apply threshold
        new_mask[np.where((pet_array >= t) & (mask_slice > 0))] = 1
        
    return new_mask

In [ ]:
import colorsys
import random

seed_color = 0
def random_colors(N, bright=True):
    """
    Generate random colors.
    To get visually distinct colors, generate them in HSV space then
    convert to RGB.
    """
    brightness = 1.0 if bright else 0.7
    hsv = [(i / N, 1, brightness) for i in range(N)]
    colors = list(map(lambda c: colorsys.hsv_to_rgb(*c), hsv))
    random.Random(seed_color).shuffle(colors)
    return colors

In [ ]:
from skimage import filters

def calculate_threshold2(roi):
#     return 2.5
    #return np.max(roi) * 0.42
    return filters.threshold_otsu(roi)


def roi2mask_slice(pet_array, mask_array):
    
    new_mask = np.zeros(mask_array.shape)
    
    for num_slice in range(mask_array.shape[0]):
        mask_slice = mask_array[num_slice]
        roi = pet_array[mask_slice > 0]
        
        threshold = calculate_threshold2(roi)

        # apply threshold
        new_mask[num_slice][np.where((pet_array >= threshold) & (mask_slice > 0))] = 1
        
    return new_mask

from skimage import measure

# def to_onehot(values):
#     n_values = np.max(values) + 1
#     return np.eye(n_values)[values]

from mrcnn import visualize
from mrcnn.visualize import display_images


In [ ]:

def plot_seg(study_uid, img_path, axis=1):
    
    
    figsize=(16, 16)
    fig, axes = plt.subplots(1, 3, figsize=figsize)
    
    pet_path = img_path['pet_path']
    mask_path = img_path['mask_path']
    
    
    pet_img = sitk.ReadImage(pet_path, dtypes['pet_img'])
    mask_img_true = sitk.ReadImage(mask_path, dtypes['mask_img'])
    
    
    pet_array_= sitk.GetArrayFromImage(pet_img)
    mask_array_true_ = sitk.GetArrayFromImage(mask_img_true)
    
    # flip axial axis
    pet_array = np.flip(pet_array_, axis=0)
    
    for ii, threshold in enumerate(['relative', 'absolute', 'otsu']):
        titles = {'absolute': '2.5 SUV', 
                  'relative': '42 % SUV_max',
                  'otsu': 'otsu'}

    
        mask_array_true = roi2mask(pet_array=pet_array_, mask_array=mask_array_true_,
                                  threshold=threshold)
        mask_array_true = np.flip(mask_array_true, axis=0)


        # MIP
        mip_pet = np.max(pet_array, axis=axis)
        mip_mask_true = np.max(mask_array_true, axis=axis)


        #
        # prepapre img for plotting 
        mip_pet[mip_pet>5.0] = 5.0
        mip_pet = (255*mip_pet/5.0).astype(int)
        # convert to RBG + MIP
        image = mip_pet[:, :, None] * np.ones(3, dtype=int)[None, None, :]

        # add dims
        mip_mask_true = np.expand_dims(mip_mask_true, axis=-1)


        # generate bounding box from the segmentation
#         bbox = []
#         for i in range(mip_mask_true.shape[2]):
#             indexes = np.where(mip_mask_true[:, :, i])
#             y1, y2 = min(indexes[0]), max(indexes[0])
#             x1, x2 = min(indexes[1]), max(indexes[1])
#             bbox.append([y1, x1, y2, x2])
        bbox = [[0, 0, mip_mask_true.shape[0], mip_mask_true.shape[1]]]

        bbox = np.array(bbox)

        colors = random_colors(bbox.shape[0])

        # plot the result
        class_ids, class_names = np.ones(bbox.shape[0], dtype=int), ["", ""] # ['background', 'lymphoma']

    #     print(image.shape, bbox.shape, mip_mask_true.shape, class_ids.shape)
    #     print(np.sum(mip_mask_true))

    #     visualize.display_instances(image, np.zeros(bbox.shape), mip_mask_pred, class_ids, class_names, show_bbox=False)

        visualize.display_instances(image, bbox, mip_mask_true, class_ids, class_names, show_bbox=False,
                                   ax=axes[ii])
        axes[ii].set_title(titles[threshold])

    plt.show()
    
    
    
    

In [ ]:
sample = df.sample(n=1)
sample

In [ ]:
print(sample.iloc[0]['STUDY_UID'])

In [ ]:
sample = df.sample(n=1)
row = sample.iloc[0]

study_uid = row['STUDY_UID']
img_path = {'pet_path': row['NIFTI_PET'],
           'mask_path': row['NIFTI_MASK']}


In [ ]:
img_path

In [ ]:
plot_seg(study_uid, img_path)
plot_seg(study_uid, img_path, axis=2)

In [ ]:
# """
# 1.2.250.1.74.20120622095500.60000021132086
# """

# for idx, row in df[df['split']=='val'].iterrows():

#     study_uid = row['STUDY_UID']
#     img_path = {'pet_path': row['NIFTI_PET'],
#                'mask_path': row['NIFTI_MASK']}
#     print(study_uid)
#     print(row['subset'])
    
#     plot_seg(study_uid, img_path)
#     plot_seg(study_uid, img_path, axis=2)

In [ ]:
# # for idx, row in df[df['STUDY_UID'].isin({'2.16.840.1.113669.632.20.310000.20002794776',
# #                                         '1.2.276.0.7230010.3.2.303858',
# #                                         '1.3.51.0.1.1.10.10.13.63.102265672.3880438'})].iterrows():

# study_uid = '1.2.840.113619.2.290.3.252365070.563.1422859091.279'

# for idx, row in df[df['STUDY_UID']==study_uid].iterrows():

#     study_uid = row['STUDY_UID']
#     img_path = {'pet_path': row['NIFTI_PET'],
#                'mask_path': row['NIFTI_MASK']}
#     print(study_uid)
#     print(row['subset'])
    
#     plot_seg(study_uid, img_path)
#     plot_seg(study_uid, img_path, axis=2)

In [ ]:
df['subset'].value_counts()

In [ ]:


for idx, row in df[df['subset']=='excluded'].iterrows():

    study_uid = row['STUDY_UID']
    img_path = {'pet_path': row['NIFTI_PET'],
               'mask_path': row['NIFTI_MASK']}
    print(study_uid)
    print(row['subset'])
    
    plot_seg(study_uid, img_path)
    plot_seg(study_uid, img_path, axis=2)
    

# display every MIP

In [ ]:
#for idx, row in df_val.sort_values(by='dice_cnn', ascending=False).head(N).iterrows()


for idx, row in df.iterrows():

    study_uid = row['STUDY_UID']
    img_path = {'pet_path': row['NIFTI_PET'],
               'mask_path': row['NIFTI_MASK']}
    print(study_uid)
    print(row['subset'])
    
    plot_seg(study_uid, img_path)
    plot_seg(study_uid, img_path, axis=2)
    

In [ ]:
# Test Seg intersection otsu | 2.5 SUV

In [ ]:

def plot_seg_otsu_absolute(study_uid, img_path, axis=1):
    
    threshold_values = [3.5, 4.0]
    
    figsize=(16, 16)
    fig, axes = plt.subplots(1, len(threshold_values)+1, figsize=figsize)
    
    pet_path = img_path['pet_path']
    mask_path = img_path['mask_path']
    
    
    pet_img = sitk.ReadImage(pet_path, dtypes['pet_img'])
    mask_img_true = sitk.ReadImage(mask_path, dtypes['mask_img'])
    
    
    pet_array_= sitk.GetArrayFromImage(pet_img)
    mask_array_true_ = sitk.GetArrayFromImage(mask_img_true)
    
    # flip axial axis
    pet_array = np.flip(pet_array_, axis=0)
    
    
    mask_otsu = roi2mask(pet_array=pet_array_, mask_array=mask_array_true_,
                                  threshold='otsu')    
    for ii, tval in enumerate(threshold_values) :

        mask_absolute = roi2mask(pet_array=pet_array_, mask_array=mask_array_true_,
                                      threshold='absolute', tval=tval)

        mask_array_true = mask_otsu * mask_absolute
        mask_array_true = np.flip(mask_array_true, axis=0)


        # MIP
        mip_pet = np.max(pet_array, axis=axis)
        mip_mask_true = np.max(mask_array_true, axis=axis)


        #
        # prepapre img for plotting 
        mip_pet[mip_pet>5.0] = 5.0
        mip_pet = (255*mip_pet/5.0).astype(int)
        # convert to RBG + MIP
        image = mip_pet[:, :, None] * np.ones(3, dtype=int)[None, None, :]

        # add dims
        mip_mask_true = np.expand_dims(mip_mask_true, axis=-1)


        # generate bounding box from the segmentation
    #         bbox = []
    #         for i in range(mip_mask_true.shape[2]):
    #             indexes = np.where(mip_mask_true[:, :, i])
    #             y1, y2 = min(indexes[0]), max(indexes[0])
    #             x1, x2 = min(indexes[1]), max(indexes[1])
    #             bbox.append([y1, x1, y2, x2])
        bbox = [[0, 0, mip_mask_true.shape[0], mip_mask_true.shape[1]]]

        bbox = np.array(bbox)

        colors = random_colors(bbox.shape[0])

        # plot the result
        class_ids, class_names = np.ones(bbox.shape[0], dtype=int), ["", ""] # ['background', 'lymphoma']

    #     print(image.shape, bbox.shape, mip_mask_true.shape, class_ids.shape)
    #     print(np.sum(mip_mask_true))

    #     visualize.display_instances(image, np.zeros(bbox.shape), mip_mask_pred, class_ids, class_names, show_bbox=False)

        visualize.display_instances(image, bbox, mip_mask_true, class_ids, class_names, show_bbox=False,
                                   ax=axes[ii+1])
        axes[ii+1].set_title('otsu|{} SUV'.format(tval))
    
    visualize.display_instances(image, bbox, np.zeros(mip_mask_true.shape), class_ids, class_names, show_bbox=False,
                           ax=axes[0])
    axes[0].set_title('MIP PET')


    
    
    

In [ ]:
for idx, row in df[df['subset']=='excluded'].iterrows():

    study_uid = row['STUDY_UID']
    img_path = {'pet_path': row['NIFTI_PET'],
               'mask_path': row['NIFTI_MASK']}
    print(study_uid)
    print(row['subset'])
    
    plot_seg_otsu_absolute(study_uid, img_path)
    plot_seg_otsu_absolute(study_uid, img_path, axis=2)
    
    plt.show()
    

In [ ]:
for idx, row in df.sample(n=50).iterrows():

    study_uid = row['STUDY_UID']
    img_path = {'pet_path': row['NIFTI_PET'],
               'mask_path': row['NIFTI_MASK']}
    print(study_uid)
    print(row['subset'])
    
    plot_seg_otsu_absolute(study_uid, img_path)
    plot_seg_otsu_absolute(study_uid, img_path, axis=2)
    
    plt.show()
    